In [1]:
!git clone https://github.com/PKU-VCL-3DV/SLAM3R.git
%cd SLAM3R

Cloning into 'SLAM3R'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 182 (delta 19), reused 16 (delta 16), pack-reused 155 (from 1)
Receiving objects: 100% (182/182), 22.96 MiB | 9.31 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/kaggle/working/SLAM3R


In [2]:
!python -V

Python 3.11.11


In [ ]:
# install torch according to your cuda version
!pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 -U
!pip install opencv-python 
!pip install -r requirements.txt -U
# optional: install additional packages to support visualization and data preprocessing
!pip install -r requirements_optional.txt -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 2.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
  Att

In [ ]:
# install XFormers according to your pytorch version, see https://github.com/facebookresearch/xformers
!pip install xformers==0.0.28.post2
# compile cuda kernels for RoPE
%cd slam3r/pos_embed/curope/
!python setup.py build_ext --inplace
!cd ../../../

In [ ]:
from slam3r.models import Image2PointsModel, Local2WorldModel
Image2PointsModel.from_pretrained('siyan824/slam3r_i2p')
Local2WorldModel.from_pretrained('siyan824/slam3r_l2w')
pass

In [ ]:
# import cv2 as cv
# import numpy as np
# import os
# import yaml
# from tqdm import tqdm


# def load_camera_params(yaml_path: str):
#     """
#     Load camera intrinsic and extrinsic parameters from a ROS-style YAML file.
#     """
#     with open(yaml_path, "r") as f:
#         data = yaml.safe_load(f)

#     cam_matrix = np.array(data["camera_matrix"]["data"]).reshape((3, 3))
#     dist_coeffs = np.array(data["distortion_coefficients"]["data"]).reshape((1, 5))
#     rect_matrix = np.array(data["rectification_matrix"]["data"]).reshape((3, 3))
#     proj_matrix = np.array(data["projection_matrix"]["data"]).reshape((3, 4))
#     return cam_matrix, dist_coeffs, rect_matrix, proj_matrix


# def rectify_image(img: np.ndarray, cam_matrix, dist_coeffs, rect_matrix, proj_matrix):
#     """
#     Apply stereo rectification and undistortion to a single image.
#     """
#     h, w = img.shape[:2]
#     image_size = (w, h)
#     mapx, mapy = cv.initUndistortRectifyMap(
#         cam_matrix, dist_coeffs, rect_matrix, proj_matrix, image_size, cv.CV_32FC1
#     )
#     return cv.remap(img, mapx, mapy, cv.INTER_LINEAR)


# def save_image(img: np.ndarray, path: str):
#     os.makedirs(os.path.dirname(path), exist_ok=True)
#     cv.imwrite(path, img)


# def split_stereo_frame(frame: np.ndarray):
#     """
#     Split a side-by-side stereo frame into (right_image, left_image).
#     左图在右边，右图在左边（按用户要求）
#     """
#     h, w = frame.shape[:2]
#     mid = w // 2
#     right_img = frame[:, :mid]
#     left_img = frame[:, mid:]
#     return left_img, right_img  # 顺序为 (左相机, 右相机)


# def extract_rectified_images_from_video(
#     video_path: str,
#     output_dir: str,
#     frame_idces,
#     calib_paths: list[str] | None = None,
# ):
#     """
#     Extract a rectified stereo pair from a specific frame in a video.
#     Saves the left and right image, with left on the right side of the stereo pair.
#     """
#     if not os.path.exists(video_path):
#         raise FileNotFoundError(f"Video not found: {video_path}")
    
#     cap = cv.VideoCapture(video_path)
#     if not cap.isOpened():
#         raise IOError(f"Failed to open video: {video_path}")
#     left_paths, right_paths = [], []
#     for frame_idx in tqdm(frame_idces):
#         cap.set(cv.CAP_PROP_POS_FRAMES, frame_idx)
#         ret, frame = cap.read()
#         # cap.release()
    
#         if not ret:
#             raise ValueError(f"Could not read frame {frame_idx} from video.")
    
#         # 分割图像：左侧是右相机，右侧是左相机
#         left_img, right_img = split_stereo_frame(frame)
    
#         # 输出路径
#         left_path = os.path.join(output_dir, f"{frame_idx:06d}.jpg")
#         # right_path = os.path.join(output_dir, f"{frame_idx:06d}_right.jpg")
    
#         # 如果提供了标定文件，进行矫正
#         if calib_paths:
#             left_calib, right_calib = calib_paths
#             camL, distL, rectL, projL = load_camera_params(left_calib)
#             camR, distR, rectR, projR = load_camera_params(right_calib)
    
#             left_img = rectify_image(left_img, camL, distL, rectL, projL)
#             right_img = rectify_image(right_img, camR, distR, rectR, projR)
    
#         save_image(left_img, left_path)
#         # save_image(right_img, right_path)
#         left_paths.append(left_path)
#         # right_paths.append(right_path)

#     return left_paths, right_paths
#     # return left_img, right_img

In [ ]:
video_path = "/kaggle/input/slam3r-paperboxtestimense/testimenta.avi"
output_dir = "data"
frame_idx = 0
calib_left = "/kaggle/input/slam3r-paperboxtestimense/left.yaml"
calib_right = "/kaggle/input/slam3r-paperboxtestimense/right.yaml"
calib_paths = [calib_left, calib_right]

In [ ]:
from  matplotlib import pyplot as plt
%matplotlib inline
def cv2_img2plt_img(cv2_img: np.ndarray)-> np.ndarray:
    if cv2_img.ndim==3: # Color image, Convert BGR to RGB
        return cv2_img[:,:,::-1] 
    else: # Grayscale image
        return cv2_img

def jshow(cv2_img: np.ndarray, size: int = 4)-> None:
    plt.figure(figsize=(size, size))
    plt.imshow(cv2_img2plt_img(cv2_img), plt.cm.gray)
    plt.show()

In [ ]:
!rm -frv data
os.makedirs(output_dir, exist_ok=True)

# left, right = extract_rectified_images_from_video(video_path, output_dir, range(0, 800, 3), calib_paths)
# cv.imwrite(f"{output_dir}/{i:06d}.png", left)

!ls data | wc

In [ ]:
vid = cv2.VideoCapture(video_path)
i = 0
while True:
    _, frame = vid.read()
    if not _: 
        print(f"{i=} image saved")
        break
    cv2.imwrite(f"data/{i:02d}.png", frame)
    
!ls data | wc

In [ ]:
# import cv2
# import os
# import numpy as np
# from  matplotlib import pyplot as plt
# %matplotlib inline
# def cv2_img2plt_img(cv2_img: np.ndarray)-> np.ndarray:
#     if cv2_img.ndim==3: # Color image, Convert BGR to RGB
#         return cv2_img[:,:,::-1] 
#     else: # Grayscale image
#         return cv2_img

# def jshow(cv2_img: np.ndarray, size: int = 4)-> None:
#     plt.figure(figsize=(size, size))
#     plt.imshow(cv2_img2plt_img(cv2_img), plt.cm.gray)
#     plt.show()
    
# # --- Paths ---
# video_path = "/kaggle/input/slam3r-paperboxtestimense/Suuicha.mp4"
# output_folder = "data"

# # --- Create output folder if it doesn't exist ---
# os.makedirs(output_folder, exist_ok=True)

# # # --- Hardcoded camera parameters from right.yaml ---
# # K = np.array([
# #     [513.49236,   0.0,     338.19594],
# #     [0.0,     514.03728, 256.53249],
# #     [0.0,       0.0,       1.0     ]
# # ])

# # D = np.array([0.694361, -0.252790, -0.023128, -0.000022, 0.000000])  # distortion coefficients
# # image_size = (640, 480)  # width, height

# # # --- Precompute undistortion maps ---
# # map1, map2 = cv2.initUndistortRectifyMap(
# #     K, D, R=np.eye(3), newCameraMatrix=K,
# #     size=image_size, m1type=cv2.CV_16SC2
# # )

# # 原始图像尺寸
# w, h = 640, 480

# fx = 390.11869
# fy = 383.49252
# cx = 328.46914
# cy = 256.1368
# focal_length = 414.95255  # 单位：像素
# baseline = 0.054          # 单位：米（请根据实际情况修改）

# # 相机内参矩阵
# K = np.array([[fx,   0.     , cx],
#               [  0.     , fy, cy ],
#               [  0.     ,   0.     ,   1.     ]])

# # 畸变参数 [k1, k2, p1, p2, k3]
# dist = np.array([0.147781, -0.143127, -0.001289, 0.000231, 0.0])

# # 获取新的相机内参（保留全部视野，alpha=1）
# new_K, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), alpha=1)

# # 生成映射矩阵
# map1, map2 = cv2.initUndistortRectifyMap(K, dist, None, new_K, (w, h), cv2.CV_16SC2)

# def lr_undist(left, right):
#     left_img_un = cv2.remap(left, map1, map2, interpolation=cv2.INTER_LINEAR)
#     right_img_un = cv2.remap(right, map1, map2, interpolation=cv2.INTER_LINEAR)
#     return left_img_un, right_img_un

# # --- Open video ---
# cap = cv2.VideoCapture(video_path)
# if not cap.isOpened():
#     print("Error: Could not open video.")
#     exit()

# frame_count = 0
# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Crop right camera view (left half)
#     frame_right, frame_left = frame[:, 0:640, :], frame[:, 640:, :]

#     # Undistort the cropped frame
#     # undistorted = cv2.remap(frame_right, map1, map2, interpolation=cv2.INTER_LINEAR)
#     left_un, right_un = lr_undist(frame_left, frame_right)
#     jshow(left_un)
#     jshow(frame_left)
#     # Save the undistorted frame
#     output_path = os.path.join(output_folder, f"frame_{frame_count:06d}.png")

#     cv2.imwrite(output_path, left_un)
#     frame_count += 1
#     break

# cap.release()
# print(f"Extracted and undistorted {frame_count} frames to '{output_folder}'")


In [ ]:
# !bash scripts/demo_wild.sh

cmd = """
#!/bin/bash


######################################################################################
# set the img_dir below to the directory of the set of images you want to reconstruct
# set the postfix below to the format of the rgb images in the img_dir
######################################################################################

TEST_DATASET="Seq_Data(img_dir='/kaggle/input/slam3r-paperboxtestimense/compet_pool', postfix='.jpg', \
img_size=224, silent=False, sample_freq=1, \
start_idx=0, num_views=-1, start_freq=1, to_tensor=True)"

######################################################################################
# set the parameters for whole scene reconstruction below
# for defination of these parameters, please refer to the recon.py
######################################################################################
TEST_NAME="paper_box_test"
KEYFRAME_STRIDE=-1     #-1 for auto-adaptive keyframe stride selection
WIN_R=5
MAX_NUM_REGISTER=10
NUM_SCENE_FRAME=10
INITIAL_WINSIZE=5
CONF_THRES_L2W=12
CONF_THRES_I2P=1.5
NUM_POINTS_SAVE=1000000

UPDATE_BUFFER_INTV=1
BUFFER_SIZE=-1       # -1 if size is not limited
BUFFER_STRATEGY="reservoir"  # or "fifo"

KEYFRAME_ADAPT_MIN=1
KEYFRAME_ADAPT_MAX=20
KEYFRAME_ADAPT_STRIDE=1

GPU_ID=-1

python recon.py \
--test_name $TEST_NAME \
--dataset "${TEST_DATASET}" \
--gpu_id $GPU_ID \
--keyframe_stride $KEYFRAME_STRIDE \
--win_r $WIN_R \
--num_scene_frame $NUM_SCENE_FRAME \
--initial_winsize $INITIAL_WINSIZE \
--conf_thres_l2w $CONF_THRES_L2W \
--conf_thres_i2p $CONF_THRES_I2P \
--num_points_save $NUM_POINTS_SAVE \
--update_buffer_intv $UPDATE_BUFFER_INTV \
--buffer_size $BUFFER_SIZE \
--buffer_strategy "${BUFFER_STRATEGY}" \
--max_num_register $MAX_NUM_REGISTER \
--keyframe_adapt_min $KEYFRAME_ADAPT_MIN \
--keyframe_adapt_max $KEYFRAME_ADAPT_MAX \
--keyframe_adapt_stride $KEYFRAME_ADAPT_STRIDE \
--save_preds

"""
with open("test_paper_box.sh", 'w') as f:
    f.write(cmd)
!ls
!cat test_paper_box.sh

In [ ]:
!time bash test_paper_box.sh

In [ ]:
!cp ./results/paper_box_test/data_recon.ply /kaggle/working/
!zip /kaggle/working/result.zip ./results -r


In [ ]:
!pwd
!ls

In [ ]:
!nvidia-smi